In [ ]:

%load_ext autoreload
%autoreload 2

In [ ]:
#Get modules
#indicators
from Incision import *
from Headcuts import *
from Erosion import *
from Invasives import *
from Bioassessment import *
from Conifer import *
from VegVigor import *
from AOP import *
from Habitat import *
from Ditches import *


# Process Indicators Post To Staging Tables

In [ ]:
#CONIFER#
from Conifer import *
#!!!No Need to update if there is no new information
#Current code grabs old data which is already in staging table
df=get_conifersez_data_sql()
readydf=process_conifer(df)
post_conifer_data(df, draft=True)

In [7]:
#AOP#
#!!! No need to update if there is no new information
#Current code grabs old data which is already in staging table
from AOP import *
#Get data from SEZ_Assessment Unit Table in sde (this indicator doesn't get updated much)
df = get_aopsez_data_sql()
#rename the sql data so that we can 
readydf = Process_aop(df)
#post_AOP_data(readydf, draft=True)

In [8]:
#BIOASSESSMENT#done
from Bioassessment import *
# from Bioassessment import *
df = get_bioassessment_data()
#clean, process, grade data
readydf = process_grade_bioassessment(df, 2023)
#post data to sde.VECTOR.stage headcut table or csv
post_bioassessment(readydf, draft=True)


In [ ]:
#INVASIVES# Update Every Year
from Invasives import *
#Get raw invasive data prior to 2024 that is stored in gdbs in F:GIS/data/Monitoring/
#df= get_invasive_data_gdb()
#Get USFS rest service data spatially joined to SEZ's
usfsdf= get_USFSinvasive_data()
#Get new invasive data from sde collect
Idf = get_combined_survey_and_invasive_data()

#clean, merge with external data/ format plant type names
df, grouped_df = merge_format_invasive(Idf, usfsdf, 2024)
#Process Data and grade/rate assessment
#Be sure to update the priority list in "F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Invasives Priority Lookup.csv"
readydf, invasive_summary, invasive_summary_priority = process_grade_invasive(df)
#post data to sde.VECTOR.stage headcut table or csv
#post_invasive(readydf, draft=True)


In [ ]:
from Invasives2 import *
#Get USFS rest service data spatially joined to SEZ's
usfsdf= get_USFSinvasive_data()
#Get new invasive data from sde collect
Idf = get_combined_survey_and_invasive_data()
#clean, merge with external data/ format plant type names
df = merge_format_prioritize_invasive(Idf, usfsdf, 2024)
#Process Data and grade/rate assessment
#Be sure to update the priority list in "F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Invasives Priority Lookup.csv"
readydf, invasive_summary, invasive_summary_priority = process_grade_invasives(df, priority_columns)
#post data to sde.VECTOR.stage headcut table or csv
#post_invasive(readydf, draft=True)

In [ ]:
#HEADCUT##Update Every Year
from Headcuts import *
#Get raw headcut data prior 2024 and earlier that is stored in gdbs in F:GIS/data/Monitoring/
#df= get_allheadcut_data()
#Get new headcut data from sde collect
df = get_combined_survey_and_headcut_data()
#clean, process, grade data
readydf = process_grade_headcut(df, 2024)
#post data to sde.VECTOR.stage headcut table or csv
post_headcut(readydf, draft=True)


In [50]:
#INCISION# Update Every Year
#Get raw incision data prior 2023 and earlier that is stored in gdbs in F:GIS/data/Monitoring/
#df= get_allincision_data()
#Get incision data from sde collect
df = get_combined_survey_and_incision_data()
#clean, process, grade data
readydf = process_grade_incision(df, 2024)
#post data to sde.VECTOR.stage incision table or csv
post_incision(readydf, draft=True)

In [ ]:
#EROSION# Update Every Year
from Erosion import *

#Get raw erosion data prior 2023 and earlier that is stored in gdbs in F:GIS/data/Monitoring/
df= get_erosion_data()

#clean, process, grade data
readydf = process_grade_erosion(df, 2024)
#post data to sde.VECTOR.stage incision table or csv
post_erosion(readydf, draft=True)

In [ ]:
#VEG VIGOR#
#Use data from SEZTable until we get new lidar data
df=get_sez_data_sql()
#df= get_oldvegvig_data()
post_Veg_data(df, draft=True)


# Staging Tables to Final Table

In [ ]:
from FormatStagingTables import *
# Staging to final
dfbanks = get_fs_data(bank_stability_url)
dfbiotic = get_fs_data(biotic_integrity_url)
dfconifer = get_fs_data(conifer_url)
dfditch = get_fs_data(ditches_url)
dfinvasive = get_fs_data(invasives_url)
dfhabitat = get_fs_data(Hab_Frag_url)
dfvegetation = get_fs_data(vegetation_url)
dfincision = get_fs_data(incision_url)
dfheadcuts = get_fs_data(headcuts_url)
dfAOP = get_fs_data(AOP_url)
dfSEZ = get_fs_data_spatial(SEZ_url)
dfheadcuts.drop(columns=['small', 'medium', 'large'], inplace=True)
#Format Staging Tables
#format biotic data average if two streams in one assessment unit
dfaveraged_biotic=average_biotic_scores(dfbiotic, 2024)
#Prep SEZ Baseline Data for assessment unit...will need to rethink if acreage changes.. or just manually change in sde
keep_columns = ['SHAPE', 'SEZ_ID', 'Feature_Type', 'SEZ_Type', 'Ownership_Primary', 'Ownership_Secondary', 'Ownership_Secondary_2', 'Ownership_Secondary_3', 'Acres', 'Comments']
#dfSEZ is assessment unit information from SDE?
dfSEZ.loc[:,keep_columns].copy()
dfSEZ['SEZ_ID']= dfSEZ['SEZ_ID'].astype(int)
#Same for meadow(large polygon) and riverine(small polygon) data drop these columns because not needed in final merge, will assign SEZ ID later
columns_to_drop = {'Year', 'SEZ_ID', 'GlobalID', 'last_edited_user', 'created_date', 'OBJECTID', 'created_user', 'last_edited_date'}

#Name dataframes so we can reference later
largepolygondata= {'dfbanks': dfbanks, 
                'dfaveraged_biotic': averaged_biotic_df,
                'dfconifer': dfconifer,
                'dfditch': dfditch,
                'dfinvasive': dfinvasive,
                'dfhabitat': dfhabitat,
                'dfvegetation': dfvegetation,
                'dfincision': dfincision,
                'dfheadcuts': dfheadcuts,
                'dfAOP': dfAOP
}


#Staging Tables Riverine/ small polygons
smallpolygondata = {'dfbanks': dfbanks, 
                'dfaveraged_biotic': averaged_biotic_df,
                'dfconifer': dfconifer,
                'dfditch': dfditch,
                'dfinvasive': dfinvasive,
                'dfhabitat': dfhabitat,
                'dfvegetation': dfvegetation,
                'dfincision': dfincision,
                'dfheadcuts': dfheadcuts,
                'dfAOP': dfAOP
}
# Process large polygon (meadow) and small polygon (riverine) data
processed_largepolygon_data = process_data(largepolygondata, lookup_dict, columns_to_drop)
processed_smallpolygon_data = process_data(smallpolygondata, lookup_riverine, columns_to_drop)